# Guidelines for Prompting


## Setup
#### Load the API key and relevant Python libaries.

In [13]:
import openai
import os
import pprint

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

In [9]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role":"user", "content":prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0 # degree of randomness of the model's output
    )
    
    return response.choices[0].message.content

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [19]:
text = f"""
You should express what you want a model to do by 
providing instructions that are as clear and 
specific as you can possibly make them. 
This will guide the model towards the desired output,  
and reduce the chances of receiving irrelevant 
or incorrect responses. Don't confuse writing a 
clear prompt with writing a short prompt.  
In many cases, longer prompts provide more clarity  
and context for the model, which can lead to 
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks
into a single sentence.
```{text}```
"""

response = get_completion(prompt)

In [20]:
len(text), len(response)

(471, 256)

In [21]:
pprint.pprint(response)

('Clear and specific instructions are essential for guiding a model towards '
 'the desired output and reducing the chances of irrelevant or incorrect '
 'responses, with longer prompts often providing more clarity and context for '
 'more detailed and relevant outputs.')


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [24]:
prompt = f"""
Generate a list of three made-up book titles along
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""

response = get_completion(prompt)

In [26]:
print(response)

[
    {
        "book_id": 1,
        "title": "The Midnight Garden",
        "author": "Elena Rivers",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Echoes of the Past",
        "author": "Nathan Black",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Whispers in the Wind",
        "author": "Samantha Reed",
        "genre": "Romance"
    }
]


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [28]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some  
water boiling. While that's happening,  
grab a cup and put a tea bag in it. Once the water is  
hot enough, just pour it over the tea bag.  
Let it sit for a bit so the tea can steep. After a  
few minutes, take out the tea bag. If you  
like, you can add some sugar or milk to taste.  
And that's it! You've got yourself a delicious  
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions,  
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,  
then simply write \"No steps provided."

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:

Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [31]:
text_2 = f"""
The sun is shining brightly today, and the birds are 
singing. It's a beautiful day to go for a  
walk in the park. The flowers are blooming, and the  
trees are swaying gently in the breeze. People  
are out and about, enjoying the lovely weather.  
Some are having picnics, while others are playing  
games or simply relaxing on the grass. It's a  
perfect day to spend time outdoors and appreciate the  
beauty of nature.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions,  
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,  
then simply write \"No steps provided."

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)


Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting
- Give successful exemples of completing tasks
- Then ask the model to complete the task

In [34]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest 
valley flows from a modest spring; the  
grandest symphony originates from a single note;  
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
pprint.pprint(response)

('<grandparent>: Resilience is like a tree that bends but does not break in '
 'the face of strong winds. It is the ability to bounce back from adversity, '
 'to persevere through challenges, and to grow stronger with each obstacle '
 "overcome. Just as a tree's roots anchor it firmly in the ground, resilience "
 'is rooted in inner strength and determination. Remember, my child, that '
 'setbacks are not the end of the road, but merely a detour on the path to '
 'success. Stay strong, stay resilient, and you will weather any storm that '
 'comes your way.')


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [35]:
text = f"""
In a charming village, siblings Jack and Jill set out on  
a quest to fetch water from a hilltop  
well. As they climbed, singing joyfully, misfortune  
struck—Jack tripped on a stone and tumbled  
down the hill, with Jill following suit.  
Though slightly battered, the pair returned home to  
comforting embraces. Despite the mishap,  
their adventurous spirits remained undimmed, and they  
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple 
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following 
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill go on a quest to fetch water from a hilltop well, but encounter misfortune along the way, yet remain adventurous and joyful.

2 - Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin, tout en restant aventureux et joyeux.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin, tout en restant aventureux et joyeux.",
  "num_names": 2
}


#### Ask for output in a specified format

In [36]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, two siblings, go on a quest to fetch water from a hilltop well but encounter misfortune along the way, yet their adventurous spirits remain undimmed.

Translation: Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin, pourtant leurs esprits aventureux restent intacts.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin, pourtant leurs esprits aventureux restent intacts.",
  "num_names": 2
}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need
 help working out the financials. 
- Land costs $100  square foot
- I can buy solar panels for $250 square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 square
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
# it should be incorrect
print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.


#### We can fix this by instructing the model to work out its own solution first.

In [38]:
prompt = f"""
Your task is to determine if the student's solution 
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution  
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution 
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help 
working out the financials. 
- Land costs $100 square foot
- I can buy solar panels for $250 square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 square 
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost for the first year of operations:
Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

So, the total cost for the first year of operations as a function of the number of square feet is 360x + 100,000.
```
Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
incorrect


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [41]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
pprint.pprint(response)

('The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush '
 'designed to provide a superior cleaning experience. It features ultra-soft '
 'bristles that are gentle on the gums and teeth, while still effectively '
 'removing plaque and debris. The toothbrush also has a slim design that makes '
 'it easy to maneuver and reach all areas of the mouth.\n'
 '\n'
 'One of the standout features of the AeroGlide UltraSlim Smart Toothbrush is '
 'its smart technology. It connects to a mobile app that tracks your brushing '
 'habits and provides personalized recommendations for improving your oral '
 'hygiene routine. The app also includes a timer to ensure you are brushing '
 'for the recommended two minutes.\n'
 '\n'
 'In addition, the toothbrush is made from durable and hygienic materials that '
 'are easy to clean and maintain. The bristles are also antimicrobial, helping '
 'to prevent the growth of bacteria on the brush.\n'
 '\n'
 'Overall, the AeroGlide UltraSlim Smart